In [1]:
import json
import time
from openai import OpenAI

client = OpenAI(api_key="")


model_privacyqa = "ft:gpt-4o-mini-2024-07-18:personal:privacyqa:BEUwmeGn" #tuned on 500 samples
model_cancerqa = "ft:gpt-4o-mini-2024-07-18:personal:cancerqa:BOS35I1P" #tuned on 500 samples
model_casehold = "ft:gpt-4.1-mini-2025-04-14:personal:casehold2:BPfk4daS" #tuned on 2000 samples

model_baseline = "gpt-4o-mini"

input_file = "casehold_500_prompts.jsonl"
output_file = "casehold_generated_responses.jsonl"
output_file_b = "chatgpt_generated_responses_b.jsonl" #vs baseline model
output_file_c = "chatgptq_privacya_generated_responses.jsonl"


def generate_multiple_responses(prompt, model_id, num_samples=5):
    responses = []
    response = client.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=1.0,
        max_tokens=1000,
        n=num_samples
    )
    for choice in response.choices:
        responses.append(choice.model_dump_json())
    return responses


def process_prompts(input_path, output_path, model):
    with open(input_path, 'r') as infile, open(output_path, 'w') as outfile:
        for line in infile:
            data = json.loads(line)
            prompt = data["messages"][1]["content"]
            if prompt:
                try:
                    completions = generate_multiple_responses(prompt, model)
                    output_data = {
                        "prompt": prompt,
                        "completions": completions
                    }
                    outfile.write(json.dumps(output_data) + "\n")
                except Exception as e:
                    print(f"Error processing prompt: {prompt}\n{e}")
                    time.sleep(1)  # Brief pause in case of rate limits

process_prompts(input_file, output_file, model_privacyqa)



In [2]:
#output count
with open(output_file, 'r') as f:
    print(sum(1 for _ in f))
     

500
